In [1]:
from fileoperations.fileoperations import getFileNamesInDir
from symbtrdataextractor import symbtrreader, extractor
import os
import json
import pandas as pd 

import pdb

In [ ]:
symbtr_cols = ['Sira', 'Kod', 'Nota53', 'NotaAE', 'Koma53', 'KomaAE', 
               'Pay', 'Payda', 'Ms', 'LNS', 'Bas', 'Soz1', 'Offset']
savefolder = os.path.join('..', '..', 'symbtr_corrections', 'usul')
symbtrs = [os.path.splitext(s)[0] for s in getFileNamesInDir('../txt', keyword='*.txt')[2]]

In [ ]:
for ii, symbTr in enumerate(symbtrs):
    print str(ii) + ': ' + symbTr
    
    symbtrtxtfile = os.path.join('../txt', symbTr + '.txt')
    symbtrmu2file = os.path.join('../mu2', symbTr + '.mu2')

    usuldict = json.load(open('./unittests/data/usul_extended.json','r'))

    savefile = os.path.join(savefolder, symbTr + '.txt')

    # extract symbtr data
    txtdata = extractor.extract(symbtrtxtfile)[0]
    mu2header = symbtrreader.readMu2Header(symbtrmu2file)[0]

    # merge
    data = extractor.merge(txtdata, mu2header)

    # get the usul variant
    variant = {}
    for vrt in usuldict[data['usul']['symbtr_slug']]['variants']:
        if vrt['mu2_name'] == data['usul']['mu2_name']:
            variant = vrt
            break

    df = pd.read_csv(symbtrtxtfile, sep='\t')
    if not df.iloc[0]['Kod'] == 51:
        # create the usul row
        # 1	51			0	0	zaman	mertebe	0	usul_symbtr_internal_id	0	usul_mu2_name	0
        # 1	51			0	0	6	4	0	90	0	Yürüksemâî (6/4)	0
        usulrow = pd.DataFrame({'Sira':1, 'Kod':51, 'Nota53':'', 'NotaAE':'', 'Koma53':0, 'KomaAE':0,
                                'Pay':int(variant['num_pulses']),'Payda':int(variant['mertebe']),'Ms':0,'Offset':0,
                                'LNS':variant['symbtr_internal_id'],'Bas':0,'Soz1':variant['mu2_name']}, index=[0])
        df2 = pd.DataFrame(df,index=range(1, df.size+1))

        for index, row in df.iterrows():       
            # change null to empty string
            for key, val in row.iteritems():
                if pd.isnull(val):
                    row[key] = ''

            # make sure that "Sira" column continues consecutively
            row['Sira'] = row['Sira']+1

            # reassign
            df.iloc[index] = row

        df_usul = pd.concat([usulrow, df], ignore_index=True)[symbtr_cols]
        df_usul.to_csv(savefile, sep='\t', index=False, encoding='utf-8')
    else:
        print symbTr + " starts with the usul row"

0: muhayyer--bozlak--sofyan--havayi_da--asaf_guven
    6.0: yı   and Çe are in the same measure!
muhayyer--bozlak--sofyan--havayi_da--asaf_guven, 59, VOCAL_SECTION does not start on a measure: 6.5625
   Unknown title target.
1: hicaz_humayun--turku--curcuna--hishisi_hancer--gaziantep
   Unknown title target.
2: beyati_araban--seyir--duyek--1--sefik_gurmeric
beyati_araban--seyir--duyek--1--sefik_gurmeric, Missing section info in lyrics.
   Unknown title target.
3: huseyni--sarki--agiraksak--bendeni_gordukce--ismet_aga
huseyni--sarki--agiraksak--bendeni_gordukce--ismet_aga, Missing section info in lyrics.
   Unknown title target.
4: nihavent--sarki--duyek--ayrilik_bukuverdi--arif_sami_toker
   Unknown title target.
5: gerdaniye--turku--nimsofyan--al_almanin--malatya
   Unknown title target.
6: ussak--turku--yuruksemai_ii--cilveloy--artvin
   Unknown title target.
7: muhayyerkurdi--sarki--nimsofyan--gonlumu_gonlune--omer_sami_gupgup
   Unknown title target.
8: mahur--sarki--duyek--hosgeld